In [9]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [10]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model="gemini-pro", convert_system_message_to_human=True)

In [128]:
master_todo_list = []
master_user_info = {}

In [161]:
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool


@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


@tool
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b


class AddTodoInput(BaseModel):
    title: str = Field(..., description="The title of the todo item.")
    description: str = Field(
        None, description="The description of the todo item.")
    due_date: str = Field(None, description="The due date of the todo item.")
    priority: int = Field(0, description="The priority of the todo item.")
    completed: bool = Field(
        False, description="Whether the todo item is completed.")
    tags: list[str] = Field(
        [], description="The tags of the todo item. Always add tags exhaustively for easier searching.")
    estimated_time: int = Field(
        0, description="The estimated time to complete the todo item in minutes.")


@tool(args_schema=AddTodoInput)
def add_todo(title: str, description: str = None, due_date: str = None, priority: int = 0, completed: bool = False, tags: list[str] = [], estimated_time: int = 0) -> dict:
    """Add a todo item. Always check if a relevant todo item already exists before adding it."""
    todo = {
        "title": title,
        "description": description,
        "due_date": due_date,
        "priority": priority,
        "completed": completed,
        "tags": tags,
        "estimated_time": estimated_time
    }
    master_todo_list.append(todo)
    return todo


class FindRelevantTodosInput(BaseModel):
    id: str = Field(None, description="The id of the todo item.")
    title: str = Field(...,
                       description="Fuzzy search for todos with this title.")
    description: str = Field(
        None, description="Fuzzy search for todos with this description.")
    due_date: str = Field(
        None, description="Todos with due dates on or before this date.")
    completed: bool = Field(
        False, description="Whether the todo item is completed.")
    tags: list[str] = Field(
        [], description="The tags of the todo item. Always add tags exhaustively for easier searching.")


@tool(args_schema=FindRelevantTodosInput)
def find_relevant_todos(id: str = None, title: str = None, description: str = None, due_date: str = None, completed: bool = False, tags: list[str] = []) -> list[dict]:
    """Find relevant todos."""
    relevant_todos = []
    for todo in master_todo_list:
        if id and todo["id"] != id:
            continue
        if title and title not in todo["title"]:
            continue
        if description and description not in todo["description"]:
            continue
        if due_date and todo["due_date"] > due_date:
            continue
        if completed and todo["completed"] != completed:
            continue
        if tags and not set(tags).issubset(todo["tags"]):
            continue
        relevant_todos.append(todo)
    return relevant_todos

class ClarifyingQuestionInput(BaseModel):
    question: str = Field(description="The clarifying question you wish to ask the user.")

@tool(args_schema=ClarifyingQuestionInput, return_direct=True)
def clarifying_question(question: str) -> str:
    """Ask a clarifying question to the user."""
    return question

class UpdateUserInfoInput(BaseModel):
    name: str = Field(None, description="The name of the user.")
    date_of_birth: str = Field(None, description="The date of birth of the user.")
    month_of_birth: str = Field(None, description="The month of birth of the user.")
    year_of_birth: str = Field(None, description="The year of birth of the user.")
    age: int = Field(None, description="The age of the user.")
    profession: str = Field(None, description="The profession of the user.")
    location: str = Field(None, description="The location of the user.")

@tool(args_schema=UpdateUserInfoInput)
def update_user_info(name: str = None, date_of_birth: str = None, month_of_birth: str = None, year_of_birth: str = None, age:int = None ,profession: str = None, location: str = None) -> dict:
    """Update details about the user."""
    user_info = {}
    if name:
        user_info["name"] = name
    if date_of_birth:
        user_info["date_of_birth"] = date_of_birth
    if month_of_birth:
        user_info["month_of_birth"] = month_of_birth
    if year_of_birth:
        user_info["year_of_birth"] = year_of_birth
    if age:
        user_info["age"] = age
    if profession:
        user_info["profession"] = profession
    if location:
        user_info["location"] = location
    master_user_info.update(user_info)
    return user_info

class LookupUserInfoInput(BaseModel):
    name: str = Field(None, description="The name of the user.")
    date_of_birth: str = Field(None, description="The date of birth of the user.")
    month_of_birth: str = Field(None, description="The month of birth of the user.")
    year_of_birth: str = Field(None, description="The year of birth of the user.")
    profession: str = Field(None, description="The profession of the user.")
    location: str = Field(None, description="The location of the user.")

@tool(args_schema=LookupUserInfoInput)
def lookup_user_info(name: str = None, date_of_birth: str = None, month_of_birth: str = None, year_of_birth: str = None, profession: str = None, location: str = None) -> dict:
    """Lookup user info."""
    user_info = {}
    if name:
        user_info["name"] = name
    if date_of_birth:
        user_info["date_of_birth"] = date_of_birth
    if month_of_birth:
        user_info["month_of_birth"] = month_of_birth
    if year_of_birth:
        user_info["year_of_birth"] = year_of_birth
    if profession:
        user_info["profession"] = profession
    if location:
        user_info["location"] = location
    return user_info

tools = [multiply, add, add_todo, find_relevant_todos, clarifying_question, update_user_info, lookup_user_info]

In [162]:
from langchain.prompts import HumanMessagePromptTemplate, SystemMessagePromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, SystemMessage, ChatMessage

sys_prompt = SystemMessagePromptTemplate.from_template(template="""You are a great chatty peronsal time organizer called Anya who also happens to be an expert life coach. Be friendly but also very inquisitive to gain more information about the user and their life in general, like their name, age, profession, what's on their mind, etc. When the user mentions some deadline or event, make sure to handle it as a todo. You have access to the following tools:

{tools}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or {tool_names}

Provide only ONE action per $JSON_BLOB, as shown:

```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}

Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation""")

prompt = ChatPromptTemplate.from_messages([
    sys_prompt,
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template(template="""{input}

{agent_scratchpad}
 (reminder to respond in a JSON blob no matter what)""")
])

In [163]:
chat_obj = {}

In [164]:
# function to serialize the chat history
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema import messages_to_dict, messages_from_dict
from langchain.memory.chat_message_histories.in_memory import ChatMessageHistory
import json


def serialize_chat_history(memory: ConversationSummaryBufferMemory, session_id: str):
    try:
        extracted_messages = memory.chat_memory.messages
        ingest_to_db = messages_to_dict(extracted_messages)
        # write to db
        chat_obj[session_id] = ingest_to_db
        return True
    except Exception as e:
        print(f"Error while serializing chat history: {e}")
        return None


def deserialize_chat_history(session_id: str):
    try:
        # retrieve from db
        ingest_to_db = chat_obj[session_id]
        retrieve_from_db = json.loads(json.dumps(ingest_to_db))
        retrieved_messages = messages_from_dict(retrieve_from_db)
        retrieved_chat_history = ChatMessageHistory(
            messages=retrieved_messages)
        retrieved_memory = ConversationSummaryBufferMemory(
            chat_memory=retrieved_chat_history, llm=llm, max_token_limit=256, return_messages=True, memory_key="chat_history")
        return retrieved_memory
    except Exception as e:
        print(f"Error while deserializing chat history: {e}")
        return None

In [165]:
from langchain.memory import ConversationSummaryBufferMemory
# check if session_id exists in chat_obj
if "sess_1" not in chat_obj:
    memory = ConversationSummaryBufferMemory(
        llm=llm, max_token_limit=256, return_messages=True, memory_key="chat_history")
else:
    memory = deserialize_chat_history("sess_1")
memory.load_memory_variables({})

{'chat_history': []}

In [166]:
# Construct the JSON agent
from langchain.agents import AgentExecutor, create_structured_chat_agent
agent = create_structured_chat_agent(llm, tools, prompt)

In [167]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [168]:
def process_query(query: str) -> str:
    resp = agent_executor.invoke(
        {"input": query, "chat_history": memory.load_memory_variables({})["chat_history"]})
    memory.save_context(
        {"input": "Hi"}, {"output": str(resp["output"])}
    )
    serialize_chat_history(memory, "sess_1")

In [169]:
process_query("Hey there!")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "clarifying_question",
  "action_input": {
    "question": "What's your name?"
  }
}
```What's your name?


> Finished chain.


In [170]:
process_query("I'm Abishek")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "update_user_info",
  "action_input": {
    "name": "Abishek"
  }
}
```{'name': 'Abishek'}Action:
```
{
  "action": "Final Answer",
  "action_input": "Nice to meet you, Abishek!"
}
```

> Finished chain.


In [171]:
process_query("I'm turning 21 this May")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "add_todo",
  "action_input": {
    "title": "Abishek's 21st birthday",
    "description": "Celebrate Abishek's 21st birthday",
    "due_date": "2024-05-01"
  }
}
```
{'title': "Abishek's 21st birthday", 'description': "Celebrate Abishek's 21st birthday", 'due_date': '2024-05-01', 'priority': 0, 'completed': False, 'tags': [], 'estimated_time': 0}Action:
```
{
  "action": "Final Answer",
  "action_input": "Got it, Abishek! I've added your birthday to your calendar."
}
```

> Finished chain.


In [172]:
process_query("My college's final year project presentation is coming up")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "add_todo",
  "action_input": {
    "title": "College final year project presentation",
    "due_date": "May 10, 2023",
    "priority": 2,
    "tags": ["college", "project", "presentation"]
  }
}
```{'title': 'College final year project presentation', 'description': None, 'due_date': 'May 10, 2023', 'priority': 2, 'completed': False, 'tags': ['college', 'project', 'presentation'], 'estimated_time': 0}Action:
```
{
  "action": "Final Answer",
  "action_input": "I've added your college final year project presentation to your calendar."
}
```

> Finished chain.
